### Logger Configuration

This project uses Python logging instead of print statements.

Why?
- Debugging large datasets
- Tracking preprocessing steps
- Production ML pipelines
- Helps identify where a failure occurred

All activities will be recorded inside:
data_preparation.log


In [1]:
import logging
import os

# Create logs folder
if not os.path.exists("logs"):
    os.makedirs("logs")

# Configure logger
logging.basicConfig(
    filename="logs/data_preparation.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger()

logger.info("==== DATA PREPARATION PIPELINE STARTED ====")

### **SECTION 1 - CLEANING TRAIN DATASET**

Objective:
- Load train.csv
- Fix datatypes
- Remove duplicates
- Handle missing values
- Save clean file as train_clean.csv

Important:
Train dataset contains the target column `Sales`, so it must be handled carefully.
We will only perform data hygiene (cleaning), not feature engineering.


In [2]:
import pandas as pd
import numpy as np

logger.info("Loading train dataset")
train = pd.read_csv("train.csv", low_memory=False)

logger.info(f"Train dataset loaded with shape: {train.shape}")
train.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


#### Inspecting the structure of **`Train`** dataset

In [3]:
train.info()
train.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1017209,1.017209e+06
unique,NaN,NaN,942,NaN,NaN,NaN,NaN,4,NaN
top,NaN,NaN,2013-01-02,NaN,NaN,NaN,NaN,0,NaN
freq,NaN,NaN,1115,NaN,NaN,NaN,NaN,986159,NaN
mean,5.584297e+02,3.998341e+00,NaN,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,NaN,1.786467e-01
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,NaN,3.830564e-01
min,1.000000e+00,1.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00
25%,2.800000e+02,2.000000e+00,NaN,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
50%,5.580000e+02,4.000000e+00,NaN,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,NaN,0.000000e+00
75%,8.380000e+02,6.000000e+00,NaN,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,NaN,0.000000e+00


#### Standardizing column names of **`Train`** dataset

In [4]:
logger.info("Standardizing column names for train dataset")
train.columns = train.columns.str.strip().str.lower().str.replace(" ", "_")
logger.info(f"Columns after cleaning: {list(train.columns)}")
list(train.columns)

['store',
 'dayofweek',
 'date',
 'sales',
 'customers',
 'open',
 'promo',
 'stateholiday',
 'schoolholiday']

#### Fixing Datatypes

In [5]:
logger.info("Fixing datatypes for train dataset")

train["date"] = pd.to_datetime(train["date"])
train["stateholiday"] = train["stateholiday"].astype(str)
train["stateholiday"] = train["stateholiday"].astype("category")
train["open"] = train["open"].fillna(1).astype(int)
train["promo"] = train["promo"].astype(int)

logger.info("Datatype conversion completed for train dataset")
train.describe()

,store,dayofweek,date,sales,customers,open,promo,schoolholiday
count,1.017209e+06,1.017209e+06,1017209,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,2014-04-11 01:30:42.846061824,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
min,1.000000e+00,1.000000e+00,2013-01-01 00:00:00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,2013-08-17 00:00:00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,2014-04-02 00:00:00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,2014-12-12 00:00:00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,2015-07-31 00:00:00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00
std,3.219087e+02,1.997391e+00,NaN,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01


#### Removing duplicates from **`Train`** dataset

In [6]:
before = len(train)
train = train.drop_duplicates()
after = len(train)

logger.info(f"Removed {before - after} duplicate rows from train dataset")
print(f"Removed {before - after} duplicate rows from train dataset")

Removed 0 duplicate rows from train dataset


#### Handling the missing values from **`Train`** dataset

In [7]:
logger.info("Handling missing values in train dataset")

missing_before = train.isnull().sum()

# Fill missing values for numeric columns with median
numeric_cols = train.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if train[col].isnull().sum() > 0:
        train[col] = train[col].fillna(train[col].median())
        logger.info(f"Filled missing values in '{col}' with median")

missing_after = train.isnull().sum()
logger.info(f"Missing values before cleaning train dataset:\n{missing_before}")
logger.info(f"Missing values after cleaning train dataset:\n{missing_after}")
print(f"Missing values before cleaning train dataset:\n{missing_before}")
print(f"Missing values after cleaning train dataset:\n{missing_after}")

Missing values before cleaning train dataset:
store            0
dayofweek        0
date             0
sales            0
customers        0
open             0
promo            0
stateholiday     0
schoolholiday    0
dtype: int64
Missing values after cleaning train dataset:
store            0
dayofweek        0
date             0
sales            0
customers        0
open             0
promo            0
stateholiday     0
schoolholiday    0
dtype: int64


#### Saving the cleaned **`Train`** dataset

In [8]:
train.to_csv("train_clean.csv", index=False)
logger.info("train_clean.csv saved successfully")
print("train_clean.csv saved successfully")

train_clean.csv saved successfully


_____________________________________________


### **SECTION 2 — CLEAN TEST DATA**
Objective:
- Load test.csv
- Fix datatypes
- Remove duplicates
- Handle missing values
- Save clean file as test_clean.csv

#### Loading **`Test`** dataset

In [9]:
logger.info("Loading test dataset")
test = pd.read_csv("test.csv", low_memory=False)
logger.info(f"Test dataset loaded with shape: {test.shape}")
print(f"Test dataset loaded with shape: {test.shape}")
test.head()

Test dataset loaded with shape: (41088, 8)


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


#### Inspecting the structure of **`Test`** dataset

In [10]:
test.info()
test.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41088 entries, 0 to 41087
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             41088 non-null  int64  
 1   Store          41088 non-null  int64  
 2   DayOfWeek      41088 non-null  int64  
 3   Date           41088 non-null  object 
 4   Open           41077 non-null  float64
 5   Promo          41088 non-null  int64  
 6   StateHoliday   41088 non-null  object 
 7   SchoolHoliday  41088 non-null  int64  
dtypes: float64(1), int64(5), object(2)
memory usage: 2.5+ MB


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41088,41077.000000,41088.000000,41088,41088.000000
unique,NaN,NaN,NaN,48,NaN,NaN,2,NaN
top,NaN,NaN,NaN,2015-09-17,NaN,NaN,0,NaN
freq,NaN,NaN,NaN,856,NaN,NaN,40908,NaN
mean,20544.500000,555.899533,3.979167,NaN,0.854322,0.395833,NaN,0.443487
std,11861.228267,320.274496,2.015481,NaN,0.352787,0.489035,NaN,0.496802
min,1.000000,1.000000,1.000000,NaN,0.000000,0.000000,NaN,0.000000
25%,10272.750000,279.750000,2.000000,NaN,1.000000,0.000000,NaN,0.000000
50%,20544.500000,553.500000,4.000000,NaN,1.000000,0.000000,NaN,0.000000
75%,30816.250000,832.250000,6.000000,NaN,1.000000,1.000000,NaN,1.000000


#### Column Standardization of **`Test`** datset

In [11]:
test.columns = test.columns.str.strip().str.lower().str.replace(" ", "_")
logger.info(f"Standardized column names for test dataset: {list(test.columns)}")
print(f"Standardized column names for test dataset: {list(test.columns)}")


Standardized column names for test dataset: ['id', 'store', 'dayofweek', 'date', 'open', 'promo', 'stateholiday', 'schoolholiday']


#### Fixing datatypes of **`Test`** dataset

In [12]:
test["date"] = pd.to_datetime(test["date"])

test["stateholiday"] = test["stateholiday"].astype(str)
test["stateholiday"] = test["stateholiday"].astype("category")

test["open"] = test["open"].fillna(1).astype(int)
test["promo"] = test["promo"].astype(int)

logger.info("Datatype conversion completed for test dataset")
test.describe()

,id,store,dayofweek,date,open,promo,schoolholiday
count,41088.000000,41088.000000,41088.000000,41088,41088.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,2015-08-24 12:00:00,0.854361,0.395833,0.443487
min,1.000000,1.000000,1.000000,2015-08-01 00:00:00,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,2015-08-12 18:00:00,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,2015-08-24 12:00:00,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,2015-09-05 06:00:00,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,2015-09-17 00:00:00,1.000000,1.000000,1.000000
std,11861.228267,320.274496,2.015481,NaN,0.352748,0.489035,0.496802


#### Handling missing values of **`Test`** dataset

In [13]:
logger.info("Handling missing values in test dataset")

missing_before = test.isnull().sum()

# Fill missing values for numeric columns with median
numeric_cols = test.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if test[col].isnull().sum() > 0:
        test[col] = test[col].fillna(test[col].median())
        logger.info(f"Filled missing values in '{col}' with median")

missing_after = test.isnull().sum()

logger.info(f"Missing values before cleaning test dataset:\n{missing_before}")
logger.info(f"Missing values after cleaning test dataset:\n{missing_after}")
print(f"Missing values before cleaning test dataset:\n{missing_before}")
print(f"Missing values after cleaning test dataset:\n{missing_after}")

Missing values before cleaning test dataset:
id               0
store            0
dayofweek        0
date             0
open             0
promo            0
stateholiday     0
schoolholiday    0
dtype: int64
Missing values after cleaning test dataset:
id               0
store            0
dayofweek        0
date             0
open             0
promo            0
stateholiday     0
schoolholiday    0
dtype: int64


#### Removing Duplicates from **`Test`** dataset

In [14]:
before = len(test)
test = test.drop_duplicates()
after = len(test)

logger.info(f"Removed {before - after} duplicate rows from test dataset")
print(f"Removed {before - after} duplicate rows from test dataset")

Removed 0 duplicate rows from test dataset


#### Saving the cleaned **`Test`** data

In [15]:
test.to_csv("test_clean.csv", index=False)
logger.info("test_clean.csv saved successfully")
print("test_clean.csv saved successfully")

test_clean.csv saved successfully


_____________________________________________

### **SECTION 3 — CLEAN STORE DATA**

Objective:
- Load store.csv
- Fix datatypes
- Remove duplicates
- Handle missing values
- Save clean file as store_clean.csv

#### Loading **`Store`** dataset

In [16]:
logger.info("Loading store dataset...")
store = pd.read_csv("store.csv")
logger.info(f"Store dataset loaded with shape: {store.shape}")
print(f"Store dataset loaded with shape: {store.shape}")
store.head()

Store dataset loaded with shape: (1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


#### Inspecting the structure of **`Store`** dataset

In [17]:
store.info()
store.describe(include="all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115.00000,1115,1115,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000,571
unique,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,a,a,NaN,NaN,NaN,NaN,NaN,NaN,"Jan,Apr,Jul,Oct"
freq,NaN,602,593,NaN,NaN,NaN,NaN,NaN,NaN,335
mean,558.00000,NaN,NaN,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573,NaN
std,322.01708,NaN,NaN,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935,NaN
min,1.00000,NaN,NaN,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000,NaN
25%,279.50000,NaN,NaN,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000,NaN
50%,558.00000,NaN,NaN,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000,NaN
75%,836.50000,NaN,NaN,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000,NaN


#### Standardizing columns of **`Store`** dataset

In [18]:
store.columns = store.columns.str.strip().str.lower().str.replace(" ", "_")
logger.info(f"Standardized column names for store dataset: {list(store.columns)}")
print(f"Standardized column names for store dataset: {list(store.columns)}")

Standardized column names for store dataset: ['store', 'storetype', 'assortment', 'competitiondistance', 'competitionopensincemonth', 'competitionopensinceyear', 'promo2', 'promo2sinceweek', 'promo2sinceyear', 'promointerval']


#### Handling categorical columns of **`Store`** dataset

In [19]:
logger.info("Handling categorical columns in store dataset")
categorical_cols = ["storetype", "assortment", "promointerval"]
for col in categorical_cols:
    store[col] = store[col].astype(str)
    store[col] = store[col].replace("nan", "None")
    store[col] = store[col].astype("category")
logger.info("Categorical columns processed for store dataset")
print("Categorical columns processed for store dataset")
store.describe()

Categorical columns processed for store dataset


,store,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


#### Competition based missing values

In [20]:
logger.info("Handling competition-related missing values")

store["competitiondistance"] = store["competitiondistance"].fillna(store["competitiondistance"].median())
store["competitionopensincemonth"] = store["competitionopensincemonth"].fillna(0)
store["competitionopensinceyear"] = store["competitionopensinceyear"].fillna(0)
logger.info("Competition-related missing values handled for store dataset")
print("Competition-related missing values handled for store dataset")
store.describe()

Competition-related missing values handled for store dataset


,store,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear
count,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,571.000000,571.000000
mean,558.00000,5396.614350,4.930942,1370.939013,0.512108,23.595447,2011.763573
std,322.01708,7654.513635,4.284924,935.467654,0.500078,14.141984,1.674935
min,1.00000,20.000000,0.000000,0.000000,0.000000,1.000000,2009.000000
25%,279.50000,720.000000,0.000000,0.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,4.000000,2006.000000,1.000000,22.000000,2012.000000
75%,836.50000,6875.000000,9.000000,2011.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


#### Promo based missing values

In [21]:
logger.info("Handling promo-related missing values")

store["promo2"] = store["promo2"].fillna(0)
store["promo2sinceweek"] = store["promo2sinceweek"].fillna(0)
store["promo2sinceyear"] = store["promo2sinceyear"].fillna(0)
store["promointerval"] = store["promointerval"].fillna("None")
logger.info("Promo-related missing values handled for store dataset")
print("Promo-related missing values handled for store dataset")
store.describe()

Promo-related missing values handled for store dataset


,store,competitiondistance,competitionopensincemonth,competitionopensinceyear,promo2,promo2sinceweek,promo2sinceyear
count,1115.00000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000,1115.000000
mean,558.00000,5396.614350,4.930942,1370.939013,0.512108,12.083408,1030.239462
std,322.01708,7654.513635,4.284924,935.467654,0.500078,15.542241,1006.038782
min,1.00000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,279.50000,720.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,558.00000,2325.000000,4.000000,2006.000000,1.000000,1.000000,2009.000000
75%,836.50000,6875.000000,9.000000,2011.000000,1.000000,22.000000,2012.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


#### Generic missing values for remaining columns

In [22]:
logger.info("Handling remaining missing values in store dataset")

missing_before = store.isnull().sum()

# Fill missing values for numeric columns with median
numeric_cols = store.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if store[col].isnull().sum() > 0:
        store[col] = store[col].fillna(store[col].median())
        logger.info(f"Filled missing values in '{col}' with median")

missing_after = store.isnull().sum()

logger.info(f"Missing values before cleaning store dataset:\n{missing_before}")
logger.info(f"Missing values after cleaning store dataset:\n{missing_after}")
print(f"Missing values before cleaning store dataset:\n{missing_before}")
print(f"Missing values after cleaning store dataset:\n{missing_after}")
logger.info("All missing values in store dataset have been handled")
print("All missing values in store dataset have been handled")

Missing values before cleaning store dataset:
store                        0
storetype                    0
assortment                   0
competitiondistance          0
competitionopensincemonth    0
competitionopensinceyear     0
promo2                       0
promo2sinceweek              0
promo2sinceyear              0
promointerval                0
dtype: int64
Missing values after cleaning store dataset:
store                        0
storetype                    0
assortment                   0
competitiondistance          0
competitionopensincemonth    0
competitionopensinceyear     0
promo2                       0
promo2sinceweek              0
promo2sinceyear              0
promointerval                0
dtype: int64
All missing values in store dataset have been handled


#### Removing duplicates from **`Store`** dataset

In [23]:
before = len(store)
store = store.drop_duplicates()
after = len(store)

logger.info(f"Removed {before - after} duplicate rows from store dataset")
print(f"Removed {before - after} duplicate rows from store dataset")

Removed 0 duplicate rows from store dataset


#### Saving updated **`Store`** dataset

In [24]:
store.to_csv("store_clean.csv", index=False)
logger.info("store_clean.csv saved successfully")

logger.info("==== DATA PREPARATION PIPELINE COMPLETED ====")
print("store_clean.csv saved successfully")

store_clean.csv saved successfully


______________________
